### Mounting drive on Colab

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


###Libraries

In [2]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 4.2 MB/s 


In [3]:
import os
import cv2
import os
import h5py
import numpy as np
import pickle
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.metrics import HammingLoss
from tensorflow_addons.metrics import F1Score
os.chdir('/gdrive/My Drive/')
from numpy import savez_compressed
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Effects of changing Batch Size



---


Experimental Setup:

Following are the details of the experimental setup for the above-mentioned experiment:

Model used: ResNet50

Note: In order to make sure even though a range of batch sizes were used for this experiment all the models were trained for the same number of steps the epochs were changed accordingly.

For Ex: 

        Batch sizes used: [12, 24, 36, 48, 60, 72, 84, 96, 108, 120]

        Number of steps: 2000

        Given the relationship:

        Number of steps = (Dataset Size/ Batch Size) * Number of Epochs

        Therefore, if: Dataset Size = 24,000

        Batch Size = 12

        Number of Steps = 24,000/12 * epochs = 2,000 * epochs

        Number of Epochs = 1

        Dataset Size = 24,000

        Batch Size = 24

        Number of Steps = 24,000/24 * epochs = 1000 * epochs

        Number of Epochs = 2

        Therefore, Epochs: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

### For ImageNet as a source

In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50

from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()


pre_trained_model = ResNet50(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = None)

#for layer in pre_trained_model.layers:
#  layer.trainable = False

In [6]:
def compile_model():
  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(2000, activation='relu')(x)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(1000, activation='relu')(x)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(500, activation='relu')(x)                  
  # Add a final softmax layer for classification
  x = layers.Dense(15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model

In [7]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [8]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [9]:
print("*********** Fitting model for 12 batch size and for 1 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 12, epochs=1, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 12 batch size and for 1 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


94/94 [==============================] - 9s 71ms/step - loss: 2.2720 - acc: 0.2150 - auc: 0.7888 - recall: 0.0247 - precision: 0.4327 - hamming_loss: 0.9753 - f1_score: 0.0434
***********test accuracy is [2.271965265274048, 0.2150000035762787, 0.7888017892837524, 0.024666666984558105, 0.4327485263347626, 0.9753333330154419, array([0.06666667, 0.        , 0.        , 0.08064517, 0.00985222,
       0.09433962, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.07373272, 0.2413793 , 0.        , 0.08372093],
      dtype=float32)] ***************
******************* Train time is 500.4122308640001 ***********
f1 score is 0.04335577543824911


In [10]:
print("*********** Fitting model for 24 batch size and for 2 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 24, epochs=2, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 24 batch size and for 2 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/2
1000/1000 [==============================] - 301s 286ms/step - loss: 1.5881 - acc: 0.4428 - auc_1: 0.9057 - recall_1: 0.2830 - precision_1: 0.6474 - hamming_loss: 0.7170 - f1_score: 0.3732 - val_loss: 3.6155 - val_acc: 0.2700 - val_auc_1: 0.7463 - val_recall_1: 0.1930 - val_precision_1: 0.3578 - val_hamming_loss: 0.8070 - val_f1_score: 0.2186
Epoch 2/2
94/94 [==============================] - 8s 66ms/step - loss: 1.0104 - acc: 0.6607 - auc_1: 0.9628 - recall_1: 0.5890 - precision_1: 0.7538 - hamming_loss: 0.4110 - f1_score: 0.6381
***********test accuracy is [1.0103740692138672, 0.6606666445732117, 0.9628415703773499, 0.5889999866485596, 0.7538396120071411, 0.41100001335144043, array([0.4042553 , 0.60714287, 0.6466166 , 0.7027027 , 0.77157354,
       0.8463612 , 0.50167227, 0.60377353, 0.6138614 , 0.7235142 ,
       0.379845  , 0.77134985, 0.884058  , 0.32520327, 0.7888889 ],
      dtype=float32)] ***************
******************* Train time is 1084.627522502 ***********
f1

In [11]:
print("*********** Fitting model for 36 batch size and for 3 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 36, epochs=3, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 36 batch size and for 3 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/3
667/667 [==============================] - 236s 328ms/step - loss: 0.6698 - acc: 0.7726 - auc_2: 0.9810 - recall_2: 0.7195 - precision_2: 0.8367 - hamming_loss: 0.2805 - f1_score: 0.7692 - val_loss: 0.8816 - val_acc: 0.7303 - val_auc_2: 0.9708 - val_recall_2: 0.6870 - val_precision_2: 0.7748 - val_hamming_loss: 0.3130 - val_f1_score: 0.7239
Epoch 2/3
667/667 [==============================] - 221s 331ms/step - loss: 0.3990 - acc: 0.8630 - auc_2: 0.9918 - recall_2: 0.8385 - precision_2: 0.8931 - hamming_loss: 0.1615 - f1_score: 0.8635 - val_loss: 1.7594 - val_acc: 0.6220 - val_auc_2: 0.9287 - val_recall_2: 0.5950 - val_precision_2: 0.6759 - val_hamming_loss: 0.4050 - val_f1_score: 0.6289
Epoch 3/3
94/94 [==============================] - 6s 67ms/step - loss: 4.0177 - acc: 0.3840 - auc_2: 0.7653 - recall_2: 0.3703 - precision_2: 0.4077 - hamming_loss: 0.6297 - f1_score: 0.3832
***********test accuracy is [4.017659664154053, 0.3840000033378601, 0.7652795314788818, 0.370333343744

In [12]:
print("*********** Fitting model for 48 batch size and for 4 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 48, epochs=4, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 48 batch size and for 4 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/4
500/500 [==============================] - 205s 376ms/step - loss: 0.3077 - acc: 0.9070 - auc_3: 0.9938 - recall_3: 0.8875 - precision_3: 0.9298 - hamming_loss: 0.1125 - f1_score: 0.9078 - val_loss: 1.0113 - val_acc: 0.7103 - val_auc_3: 0.9586 - val_recall_3: 0.6783 - val_precision_3: 0.7770 - val_hamming_loss: 0.3217 - val_f1_score: 0.7230
Epoch 2/4
500/500 [==============================] - 186s 371ms/step - loss: 0.1899 - acc: 0.9439 - auc_3: 0.9961 - recall_3: 0.9357 - precision_3: 0.9520 - hamming_loss: 0.0643 - f1_score: 0.9436 - val_loss: 0.7095 - val_acc: 0.8133 - val_auc_3: 0.9727 - val_recall_3: 0.8037 - val_precision_3: 0.8369 - val_hamming_loss: 0.1963 - val_f1_score: 0.8180
Epoch 3/4
500/500 [==============================] - 186s 372ms/step - loss: 0.1619 - acc: 0.9528 - auc_3: 0.9967 - recall_3: 0.9477 - precision_3: 0.9585 - hamming_loss: 0.0523 - f1_score: 0.9530 - val_loss: 0.5401 - val_acc: 0.8563 - val_auc_3: 0.9806 - val_recall_3: 0.8470 - val_precision_3

In [13]:
print("*********** Fitting model for 60 batch size and for 5 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 60, epochs=5, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 60 batch size and for 5 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/5
400/400 [==============================] - 184s 416ms/step - loss: 0.1788 - acc: 0.9498 - auc_4: 0.9968 - recall_4: 0.9367 - precision_4: 0.9637 - hamming_loss: 0.0632 - f1_score: 0.9500 - val_loss: 0.7767 - val_acc: 0.7993 - val_auc_4: 0.9696 - val_recall_4: 0.7810 - val_precision_4: 0.8241 - val_hamming_loss: 0.2190 - val_f1_score: 0.8013
Epoch 2/5
400/400 [==============================] - 165s 412ms/step - loss: 0.1053 - acc: 0.9721 - auc_4: 0.9980 - recall_4: 0.9697 - precision_4: 0.9753 - hamming_loss: 0.0303 - f1_score: 0.9725 - val_loss: 0.5852 - val_acc: 0.8533 - val_auc_4: 0.9777 - val_recall_4: 0.8460 - val_precision_4: 0.8671 - val_hamming_loss: 0.1540 - val_f1_score: 0.8560
Epoch 3/5
400/400 [==============================] - 165s 412ms/step - loss: 0.0877 - acc: 0.9753 - auc_4: 0.9985 - recall_4: 0.9733 - precision_4: 0.9774 - hamming_loss: 0.0267 - f1_score: 0.9753 - val_loss: 0.7878 - val_acc: 0.8403 - val_auc_4: 0.9681 - val_recall_4: 0.8330 - val_precision_4

In [14]:
print("*********** Fitting model for 72 batch size and for 6 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 72, epochs=6, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 72 batch size and for 6 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/6
334/334 [==============================] - 173s 464ms/step - loss: 0.1254 - acc: 0.9702 - auc_5: 0.9980 - recall_5: 0.9597 - precision_5: 0.9810 - hamming_loss: 0.0402 - f1_score: 0.9702 - val_loss: 1.9625 - val_acc: 0.6437 - val_auc_5: 0.9005 - val_recall_5: 0.6310 - val_precision_5: 0.6734 - val_hamming_loss: 0.3690 - val_f1_score: 0.6517
Epoch 2/6
334/334 [==============================] - 153s 459ms/step - loss: 0.0665 - acc: 0.9842 - auc_5: 0.9988 - recall_5: 0.9828 - precision_5: 0.9860 - hamming_loss: 0.0172 - f1_score: 0.9844 - val_loss: 0.5200 - val_acc: 0.8813 - val_auc_5: 0.9793 - val_recall_5: 0.8717 - val_precision_5: 0.8916 - val_hamming_loss: 0.1283 - val_f1_score: 0.8814
Epoch 3/6
334/334 [==============================] - 154s 460ms/step - loss: 0.0592 - acc: 0.9866 - auc_5: 0.9987 - recall_5: 0.9849 - precision_5: 0.9878 - hamming_loss: 0.0151 - f1_score: 0.9864 - val_loss: 0.7986 - val_acc: 0.8500 - val_auc_5: 0.9633 - val_recall_5: 0.8437 - val_precision_5

In [15]:
print("*********** Fitting model for 84 batch size and for 7 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 84, epochs=7, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 84 batch size and for 7 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/7
286/286 [==============================] - 163s 508ms/step - loss: 0.1235 - acc: 0.9744 - auc_6: 0.9975 - recall_6: 0.9645 - precision_6: 0.9834 - hamming_loss: 0.0355 - f1_score: 0.9739 - val_loss: 1.0814 - val_acc: 0.7957 - val_auc_6: 0.9477 - val_recall_6: 0.7890 - val_precision_6: 0.8070 - val_hamming_loss: 0.2110 - val_f1_score: 0.7990
Epoch 2/7
286/286 [==============================] - 143s 502ms/step - loss: 0.0472 - acc: 0.9887 - auc_6: 0.9987 - recall_6: 0.9876 - precision_6: 0.9901 - hamming_loss: 0.0124 - f1_score: 0.9888 - val_loss: 0.4762 - val_acc: 0.9103 - val_auc_6: 0.9807 - val_recall_6: 0.9080 - val_precision_6: 0.9184 - val_hamming_loss: 0.0920 - val_f1_score: 0.9130
Epoch 3/7
286/286 [==============================] - 143s 502ms/step - loss: 0.0658 - acc: 0.9894 - auc_6: 0.9989 - recall_6: 0.9885 - precision_6: 0.9902 - hamming_loss: 0.0115 - f1_score: 0.9894 - val_loss: 0.6028 - val_acc: 0.8420 - val_auc_6: 0.9794 - val_recall_6: 0.8280 - val_precision_6

In [16]:
print("*********** Fitting model for 96 batch size and for 8 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 96, epochs=8, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 96 batch size and for 8 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/8
250/250 [==============================] - 158s 561ms/step - loss: 0.0889 - acc: 0.9822 - auc_7: 0.9989 - recall_7: 0.9719 - precision_7: 0.9913 - hamming_loss: 0.0281 - f1_score: 0.9815 - val_loss: 0.5866 - val_acc: 0.8867 - val_auc_7: 0.9748 - val_recall_7: 0.8813 - val_precision_7: 0.8938 - val_hamming_loss: 0.1187 - val_f1_score: 0.8874
Epoch 2/8
250/250 [==============================] - 138s 554ms/step - loss: 0.0371 - acc: 0.9927 - auc_7: 0.9991 - recall_7: 0.9921 - precision_7: 0.9932 - hamming_loss: 0.0079 - f1_score: 0.9927 - val_loss: 0.9372 - val_acc: 0.8573 - val_auc_7: 0.9639 - val_recall_7: 0.8527 - val_precision_7: 0.8692 - val_hamming_loss: 0.1473 - val_f1_score: 0.8613
Epoch 3/8
250/250 [==============================] - 138s 553ms/step - loss: 0.0312 - acc: 0.9926 - auc_7: 0.9992 - recall_7: 0.9921 - precision_7: 0.9932 - hamming_loss: 0.0079 - f1_score: 0.9926 - val_loss: 1.1345 - val_acc: 0.7980 - val_auc_7: 0.9476 - val_recall_7: 0.7910 - val_precision_7

In [17]:
print("*********** Fitting model for 108 batch size and for 9 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 108, epochs=9, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 108 batch size and for 9 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/9
223/223 [==============================] - 151s 597ms/step - loss: 0.0723 - acc: 0.9845 - auc_8: 0.9991 - recall_8: 0.9754 - precision_8: 0.9925 - hamming_loss: 0.0246 - f1_score: 0.9839 - val_loss: 0.6480 - val_acc: 0.8860 - val_auc_8: 0.9717 - val_recall_8: 0.8830 - val_precision_8: 0.8931 - val_hamming_loss: 0.1170 - val_f1_score: 0.8886
Epoch 2/9
223/223 [==============================] - 131s 589ms/step - loss: 0.0234 - acc: 0.9947 - auc_8: 0.9994 - recall_8: 0.9944 - precision_8: 0.9950 - hamming_loss: 0.0056 - f1_score: 0.9947 - val_loss: 8.2270 - val_acc: 0.4800 - val_auc_8: 0.7749 - val_recall_8: 0.4683 - val_precision_8: 0.5007 - val_hamming_loss: 0.5317 - val_f1_score: 0.4911
Epoch 3/9
223/223 [==============================] - 132s 594ms/step - loss: 0.0244 - acc: 0.9944 - auc_8: 0.9993 - recall_8: 0.9941 - precision_8: 0.9948 - hamming_loss: 0.0059 - f1_score: 0.9945 - val_loss: 2.7284 - val_acc: 0.7157 - val_auc_8: 0.8958 - val_recall_8: 0.7107 - val_precision_8

In [18]:
print("*********** Fitting model for 120 batch size and for 10 epochs")
model = compile_model()
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("******************* Train time is", sum(cb.logs), "***********")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 120 batch size and for 10 epochs


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 148s 652ms/step - loss: 0.0862 - acc: 0.9838 - auc_9: 0.9990 - recall_9: 0.9720 - precision_9: 0.9926 - hamming_loss: 0.0280 - f1_score: 0.9822 - val_loss: 1.8571 - val_acc: 0.6713 - val_auc_9: 0.9069 - val_recall_9: 0.6547 - val_precision_9: 0.6997 - val_hamming_loss: 0.3453 - val_f1_score: 0.6895
Epoch 2/10
200/200 [==============================] - 128s 642ms/step - loss: 0.0197 - acc: 0.9957 - auc_9: 0.9994 - recall_9: 0.9954 - precision_9: 0.9961 - hamming_loss: 0.0046 - f1_score: 0.9957 - val_loss: 0.8526 - val_acc: 0.8610 - val_auc_9: 0.9668 - val_recall_9: 0.8533 - val_precision_9: 0.8749 - val_hamming_loss: 0.1467 - val_f1_score: 0.8626
Epoch 3/10
200/200 [==============================] - 128s 642ms/step - loss: 0.0316 - acc: 0.9954 - auc_9: 0.9994 - recall_9: 0.9948 - precision_9: 0.9956 - hamming_loss: 0.0052 - f1_score: 0.9952 - val_loss: 1.3233 - val_acc: 0.8373 - val_auc_9: 0.9446 - val_recall_9: 0.8360 - val_precisio